In [1]:
import pandas as pd
oof1 = pd.read_csv('oof_model3.csv') 
valid_list = oof1.sig_id.tolist()

In [2]:
import pandas as pd
for ii in [3,4,5,10,11]:
    oof1 = pd.read_csv(f'oof_model{ii}.csv') 
    oof1 = oof1[oof1.sig_id.isin(valid_list)]
    train_targets_scored = pd.read_csv('./input/lish-moa/train_targets_scored.csv')
    train_targets_scored = train_targets_scored[train_targets_scored.sig_id.isin(valid_list)]
    train_targets_scored = train_targets_scored.sort_values(by='sig_id')
    oof1 = oof1.sort_values(by='sig_id')
    sub_target_cols = train_targets_scored.columns.tolist()
    sub_target_cols.remove('sig_id')
    y_true = train_targets_scored[sub_target_cols].values
    y_pred = oof1[sub_target_cols].values
    from sklearn.metrics import log_loss
    score = 0
    for i in range(len(sub_target_cols)):
        score_ = log_loss(y_true[:, i], y_pred[:, i])
        score += score_ / len(sub_target_cols)
    print(ii,"CV log_loss ALL:", score) 


3 CV log_loss ALL: 0.015935769311927916
4 CV log_loss ALL: 0.01588402198773087
5 CV log_loss ALL: 0.015770070722981783
10 CV log_loss ALL: 0.016631585348881373
11 CV log_loss ALL: 0.015724777619773082


In [3]:
import pandas as pd
def get_oof(ii):
    oof1 = pd.read_csv(f'oof_model{ii}.csv') 
    oof1 = oof1[oof1.sig_id.isin(valid_list)]
    oof1 = oof1.sort_values(by='sig_id')
    oof1 = oof1.set_index('sig_id')
    return oof1

oof2 = get_oof(2)
oof3 = get_oof(3)
oof5 = get_oof(5)
oof10 = get_oof(10)
oof11 = get_oof(11)


In [4]:
import pandas as pd
def get_sub(ii):
    oof1 = pd.read_csv(f'submission_model{ii}.csv') 
    oof1 = oof1.sort_values(by='sig_id')
    oof1 = oof1.set_index('sig_id')
    return oof1

sub2 = get_sub(2)
sub3 = get_sub(3)
sub5 = get_sub(5)
sub10 = get_sub(10)
sub11 = get_sub(11)


In [5]:
# sub.head()

In [6]:
import datetime
import pandas as pd
from time import time
# from autograd import grad
# import autograd.numpy as np
import numpy as np
from numba import njit
from scipy.optimize import minimize, fsolve

In [7]:
y_true = pd.read_csv('./input/lish-moa/train_targets_scored.csv')
y_true = y_true[y_true.sig_id.isin(valid_list)]
y_true = y_true.sort_values(by='sig_id')
y_true = y_true.set_index('sig_id')
y_true = y_true.values


oof_dict = {
            'Model 2': f'oof_model{2}.csv',
            'Model 3': f'oof_model{3}.csv',
            'Model 5': f'oof_model{5}.csv',            
            'Model 10': f'oof_model{10}.csv',
            'Model 11': f'oof_model{11}.csv'
           }

oof = np.zeros((len(oof_dict), y_true.shape[0], y_true.shape[1]))
for ii in range(oof.shape[0]):
    oof1 = pd.read_csv(list(oof_dict.values())[ii]) 
    oof1 = oof1[oof1.sig_id.isin(valid_list)].sort_values(by='sig_id')
    oof1 = oof1.set_index('sig_id')
    oof[ii] = oof1.values

In [8]:
oof.shape

(5, 21948, 206)

In [9]:
# CPMP's logloss from https://www.kaggle.com/c/lish-moa/discussion/183010
def log_loss_numpy(y_pred):
    y_true_ravel = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = np.where(y_true_ravel == 1, - np.log(y_pred), - np.log(1 - y_pred))
    return loss.mean()

def func_numpy_metric(weights):
    oof_blend = np.tensordot(weights, oof, axes = ((0), (0)))
    return log_loss_numpy(oof_blend)

def grad_func(weights):
    oof_clip = np.clip(oof, 1e-15, 1 - 1e-15)
    gradients = np.zeros(oof.shape[0])
    for i in range(oof.shape[0]):
        a, b, c = y_true, oof_clip[i], np.zeros((oof.shape[1], oof.shape[2]))
        for j in range(oof.shape[0]):
            if j != i:
                c += weights[j] * oof_clip[j]
        gradients[i] = -np.mean((-a*b+(b**2)*weights[i]+b*c)/((b**2)*(weights[i]**2)+2*b*c*weights[i]-b*weights[i]+(c**2)-c))
    return gradients

@njit
def grad_func_jit(weights):
    oof_clip = np.minimum(1 - 1e-15, np.maximum(oof, 1e-15))
    gradients = np.zeros(oof.shape[0])
    for i in range(oof.shape[0]):
        a, b, c = y_true, oof_clip[i], np.zeros((oof.shape[1], oof.shape[2]))
        for j in range(oof.shape[0]):
            if j != i:
                c += weights[j] * oof_clip[j]
        gradients[i] = -np.mean((-a*b+(b**2)*weights[i]+b*c)/((b**2)*(weights[i]**2)+2*b*c*weights[i]-b*weights[i]+(c**2)-c))
    return gradients

In [10]:

log_loss_scores = {}
for n, key in enumerate(oof_dict.keys()):
    score_oof = log_loss_numpy(oof[n])
    log_loss_scores[key] = score_oof
    print(f'{key} CV:\t', score_oof)


Model 2 CV:	 0.01650430218101765
Model 3 CV:	 0.015935769311927916
Model 5 CV:	 0.015770070722981776
Model 10 CV:	 0.016631585348881373
Model 11 CV:	 0.01572477761977309


In [11]:
test_weights = np.array([1 / oof.shape[0]] * oof.shape[0])


In [12]:
tol = 1e-10
init_guess = [1 / oof.shape[0]] * oof.shape[0]
bnds = [(0, 1) for _ in range(oof.shape[0])]
cons = {'type': 'eq', 
        'fun': lambda x: np.sum(x) - 1, 
        'jac': lambda x: [1] * len(x)}

print('Inital Blend OOF:', func_numpy_metric(init_guess))
start_time = time()

res_scipy = minimize(fun = func_numpy_metric, 
                     x0 = init_guess, 
                     method = 'SLSQP', 
                     jac = grad_func_jit, # grad_func 
                     bounds = bnds, 
                     constraints = cons, 
                     tol = tol)
print('Optimised Weights:', res_scipy.x)

print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] Optimised Blend OOF:', res_scipy.fun)


Inital Blend OOF: 0.01565017114194376
Optimised Weights: [0.05347229 0.01524472 0.30185396 0.11718756 0.51224147]
[00:38] Optimised Blend OOF: 0.015578943781218577


In [13]:
print('Check the sum of all weights:', np.sum(res_scipy.x))
if np.sum(res_scipy.x) - 1 <= tol:
    print('Great! The sum of all weights equals to 1!')
else:
    print('Manual adjustion is needed to modify the weights.')


Check the sum of all weights: 0.9999999999999999
Great! The sum of all weights equals to 1!


In [15]:
beta = res_scipy.x

sub = sub2*beta[0]+sub3*beta[1]+sub5*beta[2]+sub10*beta[3]+sub11*beta[4]
sub.to_csv('submission.csv')